# Catalog generators

**Caden Gobat**, The George Washington University

In [ ]:
import pandas as pd
from code.utilities import split_filters
from code.xrt import XRT_lightcurve, get_photonIndex, get_temporalIndex, get_columnDensity
 
alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha += alpha.lower()
numeric = ".0123456789"

## Short GRB sample

In [ ]:
swift = pd.read_html("https://swift.gsfc.nasa.gov/archive/grb_table/fullview/")[0] # get latest Swift catalog
swift.columns = [col[0] for col in swift.columns] # reduce/flatten MultiIndex
for grb in swift[swift["TriggerNumber"].apply(pd.to_numeric,errors="coerce").isna()]["GRB"]: # non-Swift bursts
    bat_data = pd.read_html(f"https://swift.gsfc.nasa.gov/archive/grb_table/fullview/{grb}/")[0]
    T90 = bat_data.loc[bat_data[0]=="T90: c",1].values # get the T90 anyway, if it exists
    swift.loc[swift["GRB"]==grb,"BAT T90[sec]"] = pd.to_numeric(T90, errors="coerce")

In [ ]:
valid = ['050509B', '050724', '051103', '051105', '051114', '051211A', '051221', '060427B',
         '060429', '060502B', '070124', '070201', '070429B', '070714B', '070810B', '071112B',
         '080905', '080913', '081211B', '081216', '081223', '090531B', '090715A', '090916A',
         '090927A', '091117A', '091126B', '100213A', '100216A', '100816A', '110402A', '110802A',
         '120811B', '120830A', '140428B', '140604A', '150118C', '150424A', '150906B', '150922A',
         '151221A', '160111A', '160219A', '160406A', '160620A', '160709A', '160820A', '160822A',
         '160825A', '160829A', '161004A', '170127C', '170206A', '170219A', '170220A', '170222A',
         '170403A', '170616A', '170708A', '170728B', '170805A', '170805B', '170816A', '170817A',
         '170822A', '170825A', '170826A', '170827A', '170827B', '170921B', '171030A', '171103A',
         '171106A', '171223A', '180317A', '180418A', '180529A', '180618A', '180626C', '180715B',
         '180716A', '180728B', '180824A', '181121A', '181126B', '181222B', '181225A', '190121A',
         '190206A', '190331C', '190606A', '190626B', '190630A', '190719C', '190724A', '190810A',
         '190813A', '190830B', '190831B', '190903A', '190913A', '191017C', '191101B', '191116A',
         '191203A', '191221A', '200103A', '200128A', '200128B', '200129A', '200212A', '200219A',
         '200221A', '200224C', '200306B', '200307A', '200308A', '200313B', '200325A', '200327A',
         '200401A', '200405B', '200420A', '200423A', '200501A', '200506B', '200509B', '200514B',
         '200521A', '200605A', '200623B', '200626A', '200703A', '200706A', '200710A', '200714B',
         '200716C', '200718A', '200805A', '200815A', '200817A', '200824A', '200826A', '200903C',
         '200907A', '200908A', '200916B', '200920A', '200920B', '200923A', '200928A', '201103A',
         '201108A', '201109A', '201111A', '201130A', '201214B', '201221B', '201222A', '201227A',
         '210124B', '210205B', '210307B', '210326A', '210410A', '210421C', '210424B', '210425A',
         '210506A', '210510A', '210529A', '210529B', '210601A', '210605A'] # flagged from GCN Circulars

valid += ['050202', '050509B', '050709', '050724A', '050813', '050906', '050925',
          '051210', '051221A', '060121', '060313', '060502B', '060801', '061006',
          '061201', '061210', '061217', '070209', '070406', '070429B', '070707',
          '070714B', '070724A', '070729', '070809', '070810B', '071017', '071112B',
          '071227', '080121', '080123', '080426', '080503', '080702A', '080905A',
          '080919', '081024A', '081024B', '081226A', '081226B', '090305', '090305A',
          '090426', '090426A', '090510', '090515', '090607', '090621B', '090916',
          '091109B', '091117', '100117A', '100206A', '100213', '100625A', '100628A',
          '100702A', '101219A', '101224A', '110112A', '110112B', '110420B', '111020A',
          '111117A', '111121A', '111222A', '120229A', '120305A', '120521A', '120630A',
          '120804A', '120817B', '121226A', '130313A', '130515A', '130603B', '130626A',
          '130716A', '130822A', '130912A', '131004A', '131125A', '131126A', '131224A',
          '140129B', '140320A', '140402A', '140414A', '140516A', '140606A', '140619B',
          '140622A', '140903A', '140930B', '141202A', '141205A', '141212A', '150101A',
          '150101B', '150120A', '150301A'] # Fong, et al.

valid += ['130313A', '130822A', '130912A', '140903A', '141212A', '150120A', '150423A',
          '150831A', '160303A', '160408A', '160410A', '160411A', '160601A', '160612A',
          '160624A', '161001A', '170112A', '170127B', '170428A', '170524A', '180715A',
          '180718A', '180727A', '180805B', '181126A', '190427A', '191031D', '200623A',
          '201221D'] # Rastinejad, et al.

In [ ]:
sGRBs = swift[(swift["BAT T90[sec]"].apply(pd.to_numeric, errors="coerce") <= 2) | swift["GRB"].isin(valid)].copy() # filter catalog for GRBs with a valid T90 that is <2s

In [ ]:
# format columns
sGRBs.drop(['Time[UT]', 'BAT RA(J2000)', 'BAT Dec(J2000)', 'BAT 90%Error Radius[arcmin]',
            'BAT Fluence(15-150 keV)[10-7 erg/cm2]', 'BAT Fluence90% Error(15-150 keV)[10-7 erg/cm2]',
            'BAT 1-sec PeakPhoton Flux(15-150 keV)[ph/cm2/sec]', 'BAT 1-sec PeakPhoton Flux90% Error(15-150 keV)[ph/cm2/sec]',
            'BAT Photon Index(15-150 keV)(PL = simple power-law,CPL = cutoff power-law)',
            'BAT Photon Index90% Error(15-150 keV)', 'XRT 90%Error Radius[arcsec]', 'XRT Column Density(NH)[1021 cm-2]',
            'XRT Early Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'XRT 11 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]',
            'XRT 24 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'UVOT RA(J2000)', 'UVOT Dec(J2000)', 'UVOT 90%Error Radius[arcsec]',
            'Host Galaxy', 'Comments', 'References', 'Burst Advocate'],axis=1,inplace=True) # already in master catalog
sGRBs.rename(columns={'XRT Time to FirstObservation[sec]':'XRT dt [sec]','XRT InitialTemporalIndex':"α",
                      'XRTSpectral Index(Gamma)':'Γ','UVOT Time toFirst Observation[sec]':'UVOT dt [sec]',
                      'UVOT Other FilterMagnitudes':'Other UVOT Filters',"XRT RA(J2000)":"RA","XRT Dec(J2000)":"Dec"},inplace=True)

In [ ]:
optical_obs = []
for idx,entry in sGRBs["Other UVOT Filters"].iteritems():
    if pd.isna(entry):
        entries = []
    else:
        entries = split_filters(entry)
    try:
        filterdict = dict([entry.split(">") if ">" in entry else entry.split("=") if "=" in entry else None for entry in entries])
        filterdict["idx"] = idx
        optical_obs.append(filterdict)
    except:
        print("error",idx,end="; ")

for obs in optical_obs:
    sGRBs.loc[obs["idx"], "Other UVOT Filters"] = [obs.copy()]

In [ ]:
sGRBs["Redshift"] = ["".join([char for char in entry if char in numeric]) if type(entry) is str else entry for entry in sGRBs["Redshift"]]

In [ ]:
sGRBs.to_csv("./products/Swift_sGRB_catalog.csv")

## *Swift*-XRT lightcurves

In [ ]:
xrt_data = pd.DataFrame(columns=['Time', 'Tpos', 'Tneg', 'Flux', 'Fluxpos', 'Fluxneg', 'GRB'])
for i,row in sGRBs.iterrows():
    GRB_ID = row["GRB"]
    print(GRB_ID+" "*(7-len(GRB_ID)),end=": ")
    mode = None
    try:
        Gamma,mode = get_photonIndex(GRB_ID,sGRBs)
        sGRBs.loc[i,"Beta_X"] = Gamma.value - 1
        sGRBs.loc[i,"Beta_X_neg"] = Gamma.minus
        sGRBs.loc[i,"Beta_X_pos"] = Gamma.plus
        print("index ✓",end=", ")
    except:
        print("index ✗",end=", ")
    try:
        fluxdata = XRT_lightcurve(GRB_ID,sGRBs)
        xrt_data = xrt_data.append(fluxdata,ignore_index=True)
        print("lightcurve ✓",end=" ")
    except:
        print("lightcurve ✗",end=" ")
    if mode=="WT":
        print("(used WT spectrum)")
    else:
        print()
        
# account for upper limits
xrt_data.loc[xrt_data["Fluxneg"]==0, "Fluxneg"] = np.inf

In [ ]:
xrt_data.to_csv("./products/Swift_XRT_lightcurves.csv")